# Pricing of a Call Option with the Binomial Model

In [1]:
using CSV
using DelimitedFiles, DataFrames
using Statistics
import PyPlot as plt
using Plots
using GLM
using ShiftedArrays
using PyCall
using PlotlyJS

WebIO._IJuliaInit()

In [2]:
GOOG = DataFrame(CSV.File("GOOG.csv"))
BNTX = DataFrame(CSV.File("BNTX.csv"))

,Date,Open,High,Low,Close,Adj Close,Volume
,String,Float64,Float64,Float64,Float64,Float64,Int64
1,13/12/2021,267.142,287.0,264.2,278.22,278.22,4028700
2,14/12/2021,272.0,277.9,261.01,276.77,276.77,2172500
3,15/12/2021,271.78,289.7,271.51,287.05,287.05,2795300
4,16/12/2021,288.273,289.32,277.0,283.68,283.68,2091900
5,17/12/2021,279.6,287.933,271.68,287.28,287.28,2496200
6,20/12/2021,299.41,302.34,275.0,275.23,275.23,2669500
7,21/12/2021,275.4,281.98,266.6,271.46,271.46,2768800
8,22/12/2021,273.45,274.549,255.51,259.7,259.7,3092400
9,23/12/2021,258.33,259.52,250.05,255.29,255.29,2604500


In [3]:
PlotlyJS.plot(candlestick(x    =GOOG[!, "Date" ],
                 open =GOOG[!, "Open" ],
                 high =GOOG[!, "High" ],
                 low  =GOOG[!, "Low"  ],
                 close=GOOG[!, "Close"] ),
    Layout(title="GOOG Prices",
           yaxis_title="GOOG Stock")
    )

data: [
  "candlestick with fields close, high, low, open, type, and x"
]

layout: "layout with fields margin, template, title, and yaxis"

In [4]:
PlotlyJS.plot(candlestick(
        x    =BNTX[!, "Date" ],
        open =BNTX[!, "Open" ],
        high =BNTX[!, "High" ],
        low  =BNTX[!, "Low"  ],
        close=BNTX[!, "Close"] ),
    Layout(title="BioNTech Prices",
           yaxis_title="BNTX Stock")
    )

data: [
  "candlestick with fields close, high, low, open, type, and x"
]

layout: "layout with fields margin, template, title, and yaxis"

In [5]:
function Return( df )
    df[!,:Return    ] =     ( df.Close - lag(df.Close, 1) ) ./ lag(df.Close, 1)
    df[!,:Return_LOG] = log.( df.Close                      ./ lag(df.Close, 1) )
end

Return( GOOG )
Return( BNTX )
GOOG

,Date,Open,High,Low,Close,Adj Close,Volume,Return
,Date,Float64,Float64,Float64,Float64,Float64,Int64,Float64?
1,2021-12-13,2968.88,2971.25,2927.2,2934.09,2934.09,1205200,missing
2,2021-12-14,2895.4,2908.84,2844.85,2899.41,2899.41,1238900,-0.0118197
3,2021-12-15,2887.32,2950.34,2854.11,2947.37,2947.37,1364000,0.0165414
4,2021-12-16,2961.54,2971.03,2881.85,2896.77,2896.77,1370000,-0.0171679
5,2021-12-17,2854.29,2889.2,2835.76,2856.06,2856.06,2162800,-0.0140536
6,2021-12-20,2813.59,2852.21,2805.0,2848.03,2848.03,1013200,-0.00281158
7,2021-12-21,2863.0,2893.84,2834.7,2884.41,2884.41,977400,0.0127737
8,2021-12-22,2882.0,2946.06,2879.26,2938.98,2938.98,921900,0.018919
9,2021-12-23,2941.79,2971.45,2939.02,2942.85,2942.85,690900,0.00131682


In [6]:
PlotlyJS.plot( GOOG.Date, GOOG.Return,mode="markers+lines",
               Layout(title=      "GOOG Returns",
                      yaxis_title="GOOG Returns") )

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, and yaxis"

In [7]:
PlotlyJS.plot( BNTX.Date, BNTX.Return,mode="markers+lines",
               Layout(title="BioNTech Returns",
                      yaxis_title="BNTX Returns") )

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, and yaxis"

## 1-Step Binomial Tree

In [8]:
Year_Days       = 252
Interest_rate   = 0.0080286
Maturity        = 3/12
S0_GOOG         = GOOG.Close[length(GOOG.Close)]
S0_BNTX         = BNTX.Close[length(BNTX.Close)]
K_GOOG          = 2600
K_BNTX          = 140

VOL_DAILY_GOOG  = std(skipmissing(GOOG.Return))
VOL_ANNUAL_GOOG = VOL_DAILY_GOOG * sqrt(Year_Days)

VOL_DAILY_BNTX  = std(skipmissing(BNTX.Return))
VOL_ANNUAL_BNTX = VOL_DAILY_BNTX * sqrt(Year_Days)

UpDown_GOOG     = ( exp( + VOL_ANNUAL_GOOG * sqrt( Maturity ) ), exp( - VOL_ANNUAL_GOOG * sqrt( Maturity ) ) )
UpDown_BNTX     = ( exp( + VOL_ANNUAL_BNTX * sqrt( Maturity ) ), exp( - VOL_ANNUAL_BNTX * sqrt( Maturity ) ) )

q_GOOG          = ( ( 1 + Maturity * Interest_rate ) - UpDown_GOOG[2] ) / ( UpDown_GOOG[1] - UpDown_GOOG[2] )
q_BNTX          = ( ( 1 + Maturity * Interest_rate ) - UpDown_BNTX[2] ) / ( UpDown_BNTX[1] - UpDown_BNTX[2] )

PRICE_CALL_GOOG = (     q_GOOG   * max( 0, S0_GOOG * UpDown_GOOG[1] - K_GOOG )   + 
                  ( 1 - q_GOOG ) * max( 0, S0_GOOG * UpDown_GOOG[2] - K_GOOG ) ) / ( 1 + Maturity * Interest_rate )
PRICE_CALL_BNTX = (     q_BNTX   * max( 0, S0_BNTX * UpDown_BNTX[1] - K_BNTX )   + 
                  ( 1 - q_BNTX ) * max( 0, S0_BNTX * UpDown_BNTX[2] - K_BNTX ) ) / ( 1 + Maturity * Interest_rate )

println("Price of a Call Option of GOOGLE   with 3 month Maturity:\t", PRICE_CALL_GOOG)
println("Price of a Call Option of BioNTech with 3 month Maturity:\t", PRICE_CALL_BNTX)

Price of a Call Option of GOOGLE   with 3 month Maturity:	251.90037987067424
Price of a Call Option of BioNTech with 3 month Maturity:	25.393084669290637


### Multisteps-Binomial Tree

In [9]:
function CALL_PRICE(S, K, r, σ, T, N)
    Δt     = T / N
    U      = exp(σ * √Δt)
    D      = 1 / U
    R      = ( 1 + r * Δt )
    q      = (R - D) / (U - D)

    Payoff = [max(0, S * U^(N-i) * D^i - K ) for i = 0:N]
    
    return sum( Payoff[j+1] * binomial(N,j) * (1-q)^j * (q)^(N-j) for j in 0:N ) / R
end

CALL_PRICE (generic function with 1 method)

In [10]:
print( "\n", CALL_PRICE(S0_GOOG, K_GOOG, Interest_rate, VOL_ANNUAL_GOOG, Maturity, 30) )
print( "\n", CALL_PRICE(S0_BNTX, K_BNTX, Interest_rate, VOL_ANNUAL_BNTX, Maturity, 30) )


209.95074291428583
20.201281731270587

In [11]:
GOOG_CALL = [CALL_PRICE(S0_GOOG, K_GOOG, Interest_rate, VOL_ANNUAL_GOOG, Maturity, i) for i in 1:60]
BNTX_CALL = [CALL_PRICE(S0_BNTX, K_BNTX, Interest_rate, VOL_ANNUAL_BNTX, Maturity, i) for i in 1:60]
print("OK")

OK

In [12]:
REAL_CALL_GOOG = 209.6
REAL_CALL_BNTX =  21.7

21.7

In [13]:
function linescatter1()
    GOOG_PLOT = PlotlyJS.scatter(;x=1:60, y=GOOG_CALL,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_GOOG], 60), mode="lines", name="Real Value", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="Google Call Price, Maturity 3 months, Strike ~ S0, as function of number of steps",
                       yaxis_title="CALL price")
    PlotlyJS.plot([GOOG_PLOT,trace], layout)
end

linescatter1()
    

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, and yaxis"

In [14]:
function linescatter2()
    GOOG_PLOT = PlotlyJS.scatter(;x=1:60, y=BNTX_CALL,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_BNTX], 60), mode="lines", name="Real Value", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="BioNTech Call Price, Maturity 3 months, Strike ~ S0, as function of number of steps",
                       yaxis_title="CALL price")
    PlotlyJS.plot([GOOG_PLOT,trace], layout)
end

linescatter2()

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, and yaxis"